# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

<b> Описание данных  </b>  
Данные находятся в файле /datasets/Churn.csv (англ. «отток клиентов»).  

<b>Признаки  </b>

RowNumber — индекс строки в данных  
CustomerId — уникальный идентификатор клиента  
Surname — фамилия  
CreditScore — кредитный рейтинг  
Geography — страна проживания  
Gender — пол  
Age — возраст  
Tenure — сколько лет человек является клиентом банка  
Balance — баланс на счёте  
NumOfProducts — количество продуктов банка, используемых клиентом  
HasCrCard — наличие кредитной карты  
IsActiveMember — активность клиента  
EstimatedSalary — предполагаемая зарплата    

<b>Целевой признак  </b>

Exited — факт ухода клиента

## Подготовка данных

In [1]:
# Подключение необходимых библиотек и функций
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import f1_score, roc_curve, roc_auc_score
from sklearn.utils import shuffle
pd.options.mode.chained_assignment = None

In [2]:
# Присвоение исходного датасета переменной data
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
# Информация о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
# Первые 5 строк датасета
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
# Описательная статистика
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
# Проверка на дубликаты
data.duplicated().sum()

0

In [7]:
# В столбце Tenure есть пропуски. Не известно сколько лет клиент сотрудничает с банком,
# поэтому заменяю пропуски на 0
data['Tenure'] = data['Tenure'].fillna(0)
print('Количество пропусков в столбце Tenure:', data['Tenure'].isna().sum())

Количество пропусков в столбце Tenure: 0


In [8]:
# Буду работать с копией датасета
new_data = data.copy()
# Удаляю столбцы RowNumber, CustomerId, Surname, так как они не пригодятся для будущей модели
new_data = new_data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
display(new_data.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [9]:
# Уникальные значения столбцов Geography и Gender
data['Geography'].unique()
data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [10]:
# Кодирую столбцы Geography и Gender техникой OHE 
new_data = pd.get_dummies(new_data, drop_first=True)
new_data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [11]:
# Разделяю датасет на признаки и целевой признак
features = new_data.drop(['Exited'], axis=1)
target = new_data['Exited']

In [12]:
# Разделяю признаци и целеой признак на обучающую и тестовую выборки
# Выборки стратифицированы по целевому признаку, чтобы данные были равномернее распределены
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345, stratify=target
)
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(8000, 11)
(8000,)
(2000, 11)
(2000,)


In [13]:
# Разделяю обучающую выборку на обучающую и валидационную выборки
# Выборки стратифицированы по целевому признаку, чтобы данные были равномернее распределены
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345, stratify=target_train
)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(6000, 11)
(6000,)
(2000, 11)
(2000,)


In [14]:
# Стандартизирую признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [15]:
display(features_train.head())
display(features_valid.head())
features_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
5536,-0.143332,0.577533,-1.456283,-1.220573,0.797767,1,1,1.029613,0,1,1
8530,1.632702,-0.564119,-0.813711,0.435807,-0.916018,1,0,0.237986,0,0,0
1762,1.116413,-0.468981,-1.134997,1.245822,-0.916018,1,1,-0.686104,0,0,0
9090,1.643028,0.006707,0.150148,-1.220573,-0.916018,1,0,-0.391097,0,0,0
8777,-0.484083,-1.420358,-1.134997,1.421989,0.797767,1,0,-1.361559,0,1,1


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9375,0.094161,0.862946,-0.171138,0.757050,0.797767,1,0,-0.308325,1,0,1
9536,-1.031349,0.292120,1.756578,0.838881,-0.916018,1,0,-0.645029,1,0,0
6344,2.066384,0.196983,-0.492424,-1.220573,0.797767,1,0,0.498373,0,0,1
4357,1.085435,-0.278706,1.114006,-1.220573,0.797767,1,0,1.331035,0,0,1
9120,0.548495,1.814323,-0.171138,0.266421,-0.916018,0,1,-0.261760,0,1,1


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7837,2.066384,0.862946,-0.813711,-1.220573,-0.916018,1,0,1.202714,0,1,1
9505,0.011555,-1.039807,1.756578,0.052520,-0.916018,1,1,-0.443548,0,0,1
6208,0.662079,-0.468981,1.756578,-1.220573,0.797767,1,1,-0.347161,0,0,1
5642,1.364231,0.196983,-0.171138,-1.220573,-0.916018,1,0,0.568776,0,1,0
9643,-1.330797,0.196983,-0.813711,-1.220573,0.797767,1,1,0.223765,0,0,0


<b><font size=4>Вывод</font></b> 

В данном разделе были подключены необходимые библиотеки и функции, получена основная информация о датасете. Дубликатов не обнаружено, пропуски заполнены.  

Проведена кодировка столбцов Geography и Gender техникой ОНЕ.  

Датасет был разделен на признаки и целевой признак. В свою очередь признаки и целевой признак были разделены на обучающую, валидационную и тестовую стратифицированные выборки.Также числовые признаки были стандартизированны.

## Исследование задачи

In [16]:
# Обучение модели с помощью дерева решений
best_f1_tree = 0
best_depth_tree = 0
best_auc_roc = 0

for depth in range(1, 15 ,1):
    model_tree = DecisionTreeClassifier(max_depth=depth, random_state=12345,)
    model_tree.fit(features_train, target_train)
    predicted_valid_tree = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, predicted_valid_tree)
    auc_roc_tree = roc_auc_score(target_valid, model_tree.predict_proba(features_valid)[:, 1])
    
    if f1_tree > best_f1_tree:
        best_f1_tree = f1_tree
        best_depth_tree = depth
        best_auc_roc = auc_roc_tree

print('F1 для дерева решений:', best_f1_tree)
print('Максимальная глубина:', best_depth_tree)
print('AUC-ROC:', best_auc_roc)

F1 для дерева решений: 0.5817655571635311
Максимальная глубина: 8
AUC-ROC: 0.8073851972157057


In [17]:
# Обучение модели с помощью случайного леса
best_f1_forest = 0
best_depth_forest = 0
best_est_forest = 0
best_auc_roc = 0

for depth in range(1, 15, 1):
    for est in range(1, 200, 10):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predicted_valid_forest = model_forest.predict(features_valid)
        f1_forest = f1_score(target_valid, predicted_valid_forest)
        auc_roc_forest = roc_auc_score(target_valid, model_forest.predict_proba(features_valid)[:, 1])
        
        if f1_forest > best_f1_forest:
            best_f1_forest = f1_forest
            best_depth_forest = depth
            best_est_forest = est
            best_auc_roc = auc_roc_forest
            
print('F1 для случайного леса:', best_f1_forest)
print('Максимальная глубина:', best_depth_forest)
print('Число деревьев:', best_est_forest)
print('AUC-ROC:', best_auc_roc)

F1 для случайного леса: 0.5813586097946287
Максимальная глубина: 12
Число деревьев: 71
AUC-ROC: 0.8613127765670138


In [18]:
# Обучение модели с помощью логистической регрессии
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')#, class_weight='balanced')
model_log_reg.fit(features_train, target_train)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
f1_score_log_reg = f1_score(target_valid, predicted_valid_log_reg)
auc_roc_log_reg = roc_auc_score(target_valid, model_log_reg.predict_proba(features_valid)[:, 1])

print('F1 для логистической регрессии:', f1_score_log_reg)
print('AUC-ROC:', auc_roc_log_reg)

F1 для логистической регрессии: 0.3214953271028037
AUC-ROC: 0.7874237874237874


<b><font size=4>Вывод</font></b>  

В данном разделе были обучены модели: 
- дерево решений  
- случайный лес  
- логистическая регрессия
без учета дисбаланса классов на обучающей и валидационной выборке.  

Лучший результат был получен в моделях дерево решений и случайный лес. F1-мера 0,5817 и 0,5813 соответственно.

## Борьба с дисбалансом

In [19]:
# Доли ушедших и оставшихся клиентов. Доля оставшихся клинтов в 4 раза больше ушедших
# Нужно сравнять эти доли, чтобы модель обучалась корректнее
target_train.value_counts(normalize = 1)

0    0.796167
1    0.203833
Name: Exited, dtype: float64

In [20]:
# функция для увеличения выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

target_upsampled.value_counts(normalize=1)

1    0.505947
0    0.494053
Name: Exited, dtype: float64

In [21]:
# Функция для уменьшения выборки
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

target_downsampled.value_counts(normalize=1)

1    0.505999
0    0.494001
Name: Exited, dtype: float64

In [24]:
# Обучение модели дерево решений на увеличенной выборке
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=8)
model_tree.fit(features_upsampled, target_upsampled) 
predicted_valid_tree = model_tree.predict(features_valid)

print('F1 для дерева решений с увеличением выборки:', f1_score(target_valid, predicted_valid_tree))
print('AUC-ROC для дерева решений с увеличением выборки:', roc_auc_score(
    target_valid, model_tree.predict_proba(features_valid)[:, 1]
)
     )

print()

# Обучение модели дерево решений на уменьшенной выборке
model_tree = DecisionTreeClassifier(max_depth=8, random_state=12345)
model_tree.fit(features_downsampled, target_downsampled) 
predicted_valid_tree = model_tree.predict(features_valid)

print('F1 для дерева решений с уменьшением выборки:', f1_score(target_valid, predicted_valid_tree))
print('AUC-ROC для дерева решений с уменьшением выборки:', roc_auc_score(
    target_valid, model_tree.predict_proba(features_valid)[:, 1]
)
     )

F1 для дерева решений с увеличением выборки: 0.542910447761194
AUC-ROC для дерева решений с увеличением выборки: 0.7918727664490377

F1 для дерева решений с уменьшением выборки: 0.5416666666666666
AUC-ROC для дерева решений с уменьшением выборки: 0.794015895710811


In [25]:
# Обучение модели случайный лес на увеличенной выборке
model_forest = RandomForestClassifier(random_state=12345, n_estimators=71, max_depth=12)
model_forest.fit(features_upsampled, target_upsampled)
predicted_valid_forest = model_forest.predict(features_valid)
print('F1 для случайного леса с увеличением выборки:', f1_score(target_valid, predicted_valid_forest))
print('AUC-ROC для случайного леса с увеличением выборки:', roc_auc_score(
    target_valid, model_forest.predict_proba(features_valid)[:, 1]
)
     )

print()

# Обучение модели случайный лес на уменьшенной выборке
model_forest = RandomForestClassifier(random_state=12345, n_estimators=71, max_depth=12)
model_forest.fit(features_downsampled, target_downsampled)
predicted_valid_forest = model_forest.predict(features_valid)
print('F1 для случайного леса с уменьшением выборки:', f1_score(target_valid, predicted_valid_forest))
print('AUC-ROC для случайного леса с уменьшением выборки:', roc_auc_score(
    target_valid, model_forest.predict_proba(features_valid)[:, 1]
)
     )

F1 для случайного леса с увеличением выборки: 0.6203592814371257
AUC-ROC для случайного леса с увеличением выборки: 0.8619713704459467

F1 для случайного леса с уменьшением выборки: 0.5931558935361216
AUC-ROC для случайного леса с уменьшением выборки: 0.8663092985126882


In [26]:
# Обучение модели логистическая регрессия на увеличенной выборке
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')#, class_weight='balanced')
model_log_reg.fit(features_upsampled, target_upsampled)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
print('F1 для логистической регрессии с увеличением выборки:', f1_score(target_valid, predicted_valid_log_reg))
print('AUC-ROC для логистической регрессии с увеличением выборки:', roc_auc_score(
    target_valid, model_log_reg.predict_proba(features_valid)[:, 1]
)
     )

print()

# Обучение модели логистическая регрессия на уменьшенной выборке
model_log_reg = LogisticRegression(random_state=12345, solver='liblinear')#, class_weight='balanced')
model_log_reg.fit(features_downsampled, target_downsampled)
predicted_valid_log_reg = model_log_reg.predict(features_valid)
print('F1 для логистической регрессии с уменьшением выборки:', f1_score(target_valid, predicted_valid_log_reg))
print('AUC-ROC для логистической регрессии с уменьшением выборки:', roc_auc_score(
    target_valid, model_log_reg.predict_proba(features_valid)[:, 1]
)
     )


F1 для логистической регрессии с увеличением выборки: 0.5068493150684932
AUC-ROC для логистической регрессии с увеличением выборки: 0.7917995036639105

F1 для логистической регрессии с уменьшением выборки: 0.5042881646655232
AUC-ROC для логистической регрессии с уменьшением выборки: 0.7912735539854184


<b><font size=4>Вывод</font></b>  

В данном разделе дисбаланс классов был учтен и исправлен с помощью увеличения и уменьшения выборок. Увеличивал и уменьшал в 4 раза.  

F1-мера в дереве решений уменьшилась на 0,04, в логистичесской регрессии наоборот, значительно увеличилась, на 0,18.  

Но самой лучшей моделью по F1-мере является случайный лес. С уменьшением выборки мера незначительно увеличилась, на 0,01. С увеличением выборки также увеличилась, но уже на 0,04, конечно, не как у логистической регрессии, но заданный порог в 0,59 прошла.

## Тестирование модели

In [ ]:
# Для лучшего обучения объединяю обучающую и валидационную выборки
features_train_valid = pd.concat([features_train] + [features_valid])
target_train_valid = pd.concat([target_train] + [target_valid]) 

# Увеличиваю выборку
features_tv_upsampled, target_tv_upsampled = upsample(features_train_valid, target_train_valid, 4)

In [ ]:
# Обучаю модель случайный лес на увеличенных выборках
model_forest = RandomForestClassifier(random_state=12345, n_estimators=71, max_depth=12)
model_forest.fit(features_tv_upsampled, target_tv_upsampled)
predicted_test_forest = model_forest.predict(features_test)
f1_score_forest = f1_score(target_test, predicted_test_forest)
if f1_score_forest > 0.59:
    print('Это успех! F1 =', f1_score_forest)
    print('AUC-ROC для случайного леса на тестовой выборке:', roc_auc_score(
    target_test, model_forest.predict_proba(features_test)[:, 1]
    )
         )
else:
    print('Нужно поработать')

## Общий вывод  

В данной работе были подготовленны данные:  
- закодированны столбцы Geography и Gender  
- датасет разделен на обучающую, вылидационную и тестовую выборки  
- числовые признаки стандартизированы.  

Обучены три модели без учета дисбаланса классов. Лучшие результаты показали модели дерево решений и случайный лес.  

Для избавления от дисбаланса выборки увеличивались и уменьшались. Лучший результат был получен в модели случайный лес, F1-мера - 0,62, но логистическая регрессия приятно удивила ростом F1-меры на 0,18 до 0,5.

И, конечно, модель была протестирована на объединенной (соединил обучающую и валидационную выборки для улучшения обучения модели) увеличенной выборке. Модель обучилась еще лучше, F1-мера снова увеличилась и составила 0,6376, что больше, заявленного в задании, порога в 0,59.